In [8]:
# Libraries for parsing data
import os
import random
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup

In [2]:
# %conda install lxml

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/pytorch_p310

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    cairo-1.18.0               |       hbb29018_2         961 KB  conda-forge
    dav1d-1.2.1                |       hd590300_0         742 KB  conda-forge
    ffmpeg-7.1.0               | gpl_he2fd91e_701         9.8 MB  conda-forge
    font-ttf-ubuntu-0.83       |       h77eed37_3         1.5 MB  conda-forge
    fontconfig-2.15

libopenvino-intel-cp | 11.5 MB   | 2                                     |   1% [A
ffmpeg-7.1.0         | 9.8 MB    | 7                                     |   2% 

libopenvino-intel-gp | 8.5 MB    | #############2                        |  36% 


libopenvino-2024.4.0 | 5.1 MB    | #                                     |   3% 



libopenvino-intel-cp | 11.5 MB   | #####                                 |  14% 
ffmpeg-7.1.0         | 9.8 MB    | ##########2                           |  28% 


libopenvino-2024.4.0 | 5.1 MB    | #################                     |  46% 



libglib-2.82.2       | 3.7 MB    | ##################################6   |  94% 

libopenvino-intel-gp | 8.5 MB    | ##########################5           |  72% 
libopenvino-intel-cp | 11.5 MB   | ############2                         |  33% 


libopenvino-2024.4.0 | 5.1 MB    | ###################################6  |  96% 




aom-3.9.1            | 2.6 MB    | 2                                     |   1% 
libopenv

libopenvino-paddle-f | 641 KB    | ##################################### | 100% 



















openh264-2.4.1       | 718 KB    | ##################################### | 100% 



















openh264-2.4.1       | 718 KB    | ##################################### | 100% 






















 ... (more hidden) ...






















 ... (more hidden) ...






















                                                                                
                                                                                

                                                                                


                                                                                



                                                                                




                                                                                





                                                                                






                   

In [38]:
# Function to strip html tags from text portion
def strip_html_tags(text):
    stripped = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()
    return stripped

In [40]:
def getxmlcontent(corpus_path, file, strip_html=True):
    try:
        goid = None
        title = None
        date = None
        publisher = None
        text = None
        tree = etree.parse(corpus_path + file)
        root = tree.getroot()
        
        if root.find(".//RawLang").text == "English":
            if root.find('.//GOID') is not None:
                goid = root.find('.//GOID').text
            if root.find('.//Title') is not None:
                title = root.find('.//Title').text
            if root.find('.//NumericDate') is not None:
                date = root.find('.//NumericDate').text
            if root.find('.//PublisherName') is not None:
                publisher = root.find('.//PublisherName').text
            if root.find('.//FullText') is not None:
                text = root.find('.//FullText').text
            elif root.find('.//HiddenText') is not None:
                text = root.find('.//HiddenText').text
            elif root.find('.//Text') is not None:
                text = root.find('.//Text').text

            # Strip html from text portion
            if text is not None and strip_html == True:
                text = strip_html_tags(text)

    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
    
    return goid, title, date, publisher, text

In [44]:
def prepare_files(corpus, sample_size = None):
    corpus_path = f"/home/ec2-user/SageMaker/data/{corpus}/"
    files = random.sample(os.listdir(corpus_path), sample_size)
    rows = []

    for file in files:
        goid, title, date, publisher, text = getxmlcontent(corpus_path, file, strip_html=True)
        rows.append({
            "goid": goid,
            "title": title,
            "date": date,
            "publisher": publisher,
            "text": text
        })
    
    return pd.DataFrame(rows)

In [49]:
corpora = [
#     "dollar-tree",
#     "lululemon",
#     "ulta",
#     "walgreens",
    "walmart"
]

for corpus in corpora:
    df = prepare_files(corpus, 100)
    df.to_csv(f"output_files/{corpus}.csv")

In [ ]:
# !aws s3 cp $data_to_export s3://pq-tdm-studio-results/tdm-ale-data/1876/results/